<a href="https://colab.research.google.com/github/timofeyegorov/test/blob/main/%D0%92%D0%B5%D1%80%D1%81%D0%B8%D1%8F_%D0%A2%D1%80%D0%B0%D1%84%D0%B8%D0%BA_24_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras import utils
from sklearn.cluster import KMeans # Импортируем библиотуке KMeans для кластеризации
import time
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Таблица интенсив
intensive = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRTVUBn44-1Ymsbb8eT2jn-PlX5z5rQmIhMSPMIyo5jNyywKIOh0zasWnr3F0rI-pL7S99wZNwN3PhJ/pub?gid=800532813&single=true&output=csv'

# Таблица анкета
anketa = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQ0OjIt3L0IeUQ_y9XU4J0U891AheXAoI3CnDbAfJz8VIwVrDLXiyjlGJm5Bf9JBxzcgRAVJTE-b5S3/pub?gid=647263146&single=true&output=csv'

# Путь для сохранения таблиц на гугл диске
save_path = '/content/drive/MyDrive/Colab Notebooks/Определение источника трафика/Отчеты csv/'

# **Функции**

In [ ]:
# Функция фильтрации датасета по дате
def filter_df(start_date = '0', end_date = '0'):
  if start_date == '0':
    start_date = min(result['sent'])
  if end_date == '0':
    end_date = max(result['sent'])
  filtered_result = result[(result['sent'] >= start_date) & (result['sent'] <= end_date)]
  return filtered_result

In [ ]:
# Функция формирования двух датасетов по выбранной категории и соотвествующему столбцу в которой эти категории находятся
def channels_table(df, category, column_name):
  '''
    Функция формирования двух датасетов по выбранной категории и соотвествующему столбцу в которой эти категории находятся.
    Вход:
        category - список подкатегорий по выбранной категории
        column_name - название колонки в датасете, которая содержит названия подкатегорий
    Выход:
        df_category - датасет с разбивкой заявок трафикологов по категориям в %
        df_category_val - датасет с разбивкой заявок трафикологов по категориям в штуках
  '''

  if category == countries: start, end = 0, 3
  if category == ages: start, end = 2, 6
  if category == jobs: start, end = 0, 4
  if category == earnings: start, end = 2, 5
  if category == trainings: start, end = 0, 2
  if category == times: start, end = 1, 3

  data_category = np.zeros((len(category)+1, len(created_columns))).astype('int')
  df_category = pd.DataFrame(data_category, columns=created_columns)
  df_category.insert(0, 'Все', 0)
  df_category.insert(0, column_name, 0)

  df_category_val = pd.DataFrame(data_category, columns=created_columns)
  df_category_val.insert(0, 'Все', 0)
  df_category_val.insert(0, column_name, 0)

  for idx, el in enumerate(category):
    df_category.loc[idx, column_name] = el
    df_category_val.loc[idx, column_name] = el
    for traff_value in df['trafficologist'].unique():

      try:
        df_category.loc[idx, traff_value] = round(((df[(df[column_name]==el) & (df['trafficologist']==traff_value)].shape[0])/(df[df['trafficologist']==traff_value].shape[0])*100))
        df_category_val.loc[idx, traff_value] = round(((df[(df[column_name]==el) & (df['trafficologist']==traff_value)].shape[0])))
      except ZeroDivisionError:
        df_category.loc[idx, traff_value] = 0
        df_countries_val.loc[idx, traff_value] = 0

      df_category.loc[idx, 'Все'] += round(((df[(df[column_name]==el) & (df['trafficologist']==traff_value)].shape[0]/(df.shape[0] - df['trafficologist'].isna().sum()))*100),2)
      df_category_val.loc[idx, 'Все'] += df[(df[column_name]==el) & (df['trafficologist']==traff_value)].shape[0]

    for traff_value in df['account'].unique():

      try:
        df_category.loc[idx, traff_value] = round(((df[(df[column_name]==el) & (df['account']==traff_value)].shape[0])/(df[df['account']==traff_value].shape[0])*100))
        df_category_val.loc[idx, traff_value] = round(((df[(df[column_name]==el) & (df['account']==traff_value)].shape[0])))
      except ZeroDivisionError:
        df_category.loc[idx, traff_value] = 0
        df_countries_val.loc[idx, traff_value] = 0

  df_category.loc[idx+1, column_name] = 'ЦА'
  df_category_val.loc[idx+1, column_name] = 'ЦА'
  for i in range(1, df_category.shape[1]):
    df_category.iloc[idx+1, i] = df_category.iloc[start:end, i].sum()
    df_category_val.iloc[idx+1, i] = df_category_val.iloc[start:end, i].sum()

  df_category.drop(columns=delete_columns,inplace=True)
  df_category_val.drop(columns=delete_columns,inplace=True)
  return df_category, df_category_val

In [ ]:
# Функция формирования двух датасетов по ЦА
def target_audience_channels_table(df):
  data_ta = np.zeros((9, len(created_columns))).astype('int')
  df_ta = pd.DataFrame(data_ta, columns=created_columns)
  df_ta_val = pd.DataFrame(data_ta, columns=created_columns)

  df_ta.insert(0, 'Все', 0)
  df_ta_val.insert(0, 'Все', 0)

  for traff_value in df['trafficologist'].unique():
    for i in range(df[df['trafficologist']==traff_value].shape[0]):
      target = 0
      if df[df['trafficologist']==traff_value].iloc[i, 0] in ['Россия', 'Европа', 'Северная Америка']:
        target += 1
      if df[df['trafficologist']==traff_value].iloc[i, 1] in ['23 - 30', '30 - 40', '40 - 50', '50 - 60']:
        target += 1
      if df[df['trafficologist']==traff_value].iloc[i, 2] in ['IT сфера (разработчик, тестировщик, администратор и т.п.)',
                                                              'Профессии связанные с числами (аналитик, бухгалтер, инженер и т.п.)',
                                                              'Гуманитарий (общение с людьми, искусство, медицина и т.д.)',
                                                              'Предприниматель, руководитель']:
        target += 1
      if df[df['trafficologist']==traff_value].iloc[i, 3] in ['до 60 000 руб.', 'до 100 000 руб.', 'более 100 000 руб.']:
        target += 1
      if df[df['trafficologist']==traff_value].iloc[i, 4] in ['Да, если я точно найду работу после обучения.', 'Да, если это поможет мне в реализации проекта.']:
        target += 1
      if df[df['trafficologist']==traff_value].iloc[i, 5] in ['до 10 часов в неделю', 'более 10 часов в неделю']:
        target += 1

      df_ta.loc[target, traff_value] += round(((1/df[df['trafficologist']==traff_value].shape[0])*100),2)
      df_ta.loc[target, 'Все'] += round(((1/(df.shape[0]))*100),2)

      df_ta_val.loc[target, traff_value] += 1
      df_ta_val.loc[target, 'Все'] += 1


      if (target == 5) or (target == 6):

        df_ta.loc[7:8, traff_value] += round(((1/df[df['trafficologist']==traff_value].shape[0])*100),2)
        df_ta.loc[7:8, 'Все'] += round(((1/(df.shape[0]))*100),2)

        df_ta_val.loc[7:8, traff_value] += 1
        df_ta_val.loc[7:8, 'Все'] += 1

      if (target == 4): 
        df_ta.loc[8, traff_value] += round(((1/df[df['trafficologist']==traff_value].shape[0])*100),2)
        df_ta.loc[8, 'Все'] += round(((1/(df.shape[0]))*100),2)

        df_ta_val.loc[8, traff_value] += 1
        df_ta_val.loc[8, 'Все'] += 1 

  for traff_value in df['account'].unique():
    for i in range(df[df['account']==traff_value].shape[0]):
      target = 0
      if df[df['account']==traff_value].iloc[i, 0] in ['Россия', 'Европа', 'Северная Америка']:
        target += 1
      if df[df['account']==traff_value].iloc[i, 1] in ['23 - 30', '30 - 40', '40 - 50', '50 - 60']:
        target += 1
      if df[df['account']==traff_value].iloc[i, 2] in ['IT сфера (разработчик, тестировщик, администратор и т.п.)',
                                                              'Профессии связанные с числами (аналитик, бухгалтер, инженер и т.п.)',
                                                              'Гуманитарий (общение с людьми, искусство, медицина и т.д.)',
                                                              'Предприниматель, руководитель']:
        target += 1
      if df[df['account']==traff_value].iloc[i, 3] in ['до 60 000 руб.', 'до 100 000 руб.', 'более 100 000 руб.']:
        target += 1
      if df[df['account']==traff_value].iloc[i, 4] in ['Да, если я точно найду работу после обучения.', 'Да, если это поможет мне в реализации проекта.']:
        target += 1
      if df[df['account']==traff_value].iloc[i, 5] in ['до 10 часов в неделю', 'более 10 часов в неделю']:
        target += 1

      df_ta.loc[target, traff_value] += round(((1/df[df['account']==traff_value].shape[0])*100),2)
      df_ta_val.loc[target, traff_value] += 1

      if (target == 5) or (target == 6):
        df_ta.loc[7:8, traff_value] += round(((1/df[df['account']==traff_value].shape[0])*100),2)
        df_ta_val.loc[7:8, traff_value] += 1

      if (target == 4): 
        df_ta.loc[8, traff_value] += round(((1/df[df['account']==traff_value].shape[0])*100),2)
        df_ta_val.loc[8, traff_value] += 1

  df_ta.drop(columns=delete_columns,inplace=True)
  df_ta_val.drop(columns=delete_columns,inplace=True)

  df_ta = df_ta.round(0).astype(int)
  return df_ta, df_ta_val

In [ ]:
def getParameterCityVect(arg):
  if arg == countries[0]:
    outClass = 0
  if arg == countries[1]:
    outClass = 1  
  if arg == countries[2]:
    outClass = 2
  if arg == countries[3]:
    outClass = 3  
  if arg == countries[4]:
    outClass = 4  
  return list(utils.to_categorical(outClass, len(countries)).astype('int'))


def getParameterAgeVect(arg):
  if arg == ages[0]:
    outClass = 0
  if arg == ages[1]:
    outClass = 1  
  if arg == ages[2]:
    outClass = 2
  if arg == ages[3]:
    outClass = 3  
  if arg == ages[4]:
    outClass = 4  
  if arg == ages[5]:
    outClass = 5    
  if arg == ages[6]:
    outClass = 6      
  return list(utils.to_categorical(outClass, len(ages)).astype('int'))


def getParameterJobVect(arg):
  if arg == jobs[0]:
    outClass = 0
  if arg == jobs[1]:
    outClass = 1  
  if arg == jobs[2]:
    outClass = 2
  if arg == jobs[3]:
    outClass = 3  
  if arg == jobs[4]:
    outClass = 4  
  if arg == jobs[5]:
    outClass = 5    
  if arg == jobs[6]:
    outClass = 6    
  if arg == jobs[7]:
    outClass = 7            
  return list(utils.to_categorical(outClass, len(jobs)).astype('int'))


def getParameterEarningsVect(arg):
  if arg == earnings[0]:
    outClass = 0
  if arg == earnings[1]:
    outClass = 1  
  if arg == earnings[2]:
    outClass = 2
  if arg == earnings[3]:
    outClass = 3  
  if arg == earnings[4]:
    outClass = 4  
  return list(utils.to_categorical(outClass, len(earnings)).astype('int'))


def getParameterTrainingVect(arg):
  if arg == trainings[0]:
    outClass = 0
  if arg == trainings[1]:
    outClass = 1  
  if arg == trainings[2]:
    outClass = 2
  return list(utils.to_categorical(outClass, len(trainings)).astype('int'))


def getParameterTimeVect(arg):
  if arg == times[0]:
    outClass = 0
  if arg == times[1]:
    outClass = 1  
  if arg == times[2]:
    outClass = 2
  return list(utils.to_categorical(outClass, len(times)).astype('int'))


def getAllParameters(val):
  city = getParameterCityVect(val[0])             
  age = getParameterAgeVect(val[1])             
  job = getParameterJobVect(val[2])         
  earning = getParameterEarningsVect(val[3])             
  training = getParameterTrainingVect(val[4]) 
  times = getParameterTimeVect(val[5])   

  out = []
  out += city
  out += age
  out += job
  out += earning
  out += training
  out += times

  return out


def get01Data(values):
  xTrain = []                   # Создаем пустой xTrain

  for val in values:            # Пробегаем по всем записям базы
      x = getAllParameters(val) # Получаем полный набор данных о текущей записи val
      xTrain.append(x)          # Добавляем полученные данные в xTrain

  xTrain = np.array(xTrain)     # Переводим в numpy
  return xTrain

In [ ]:
def addLabels(xTrain01):
  xTrain01 = xTrain01.tolist()

  for i in range(len(xTrain01)):
    category = 0
    if sum(xTrain01[i][:3]):
      category += 1
    if sum(xTrain01[i][7:11]):
      category += 1
    if sum(xTrain01[i][13:17]):
      category += 1
    if sum(xTrain01[i][23:26]):
      category += 1
    if sum(xTrain01[i][26:27]):
      category += 1
    if sum(xTrain01[i][30:32]):
      category += 1

    xTrain01[i].append(category)  
  xTrain01 = np.array(xTrain01)
  return xTrain01

In [ ]:
'''
  Функция печати информации о кластере
    Входные данные:
      - x - номер кластера
'''
def printCluster(x):
  print("Размер кластера:", xTrain01[xTrain01[:, 31]==x].shape[0]) # Выведем количество элементов в кластере
  
  mX = np.mean(xTrain01[xTrain01[:, 31]==x], axis=0)  # Считаем среднее значение по кластеру
  sX = np.sum(xTrain01[xTrain01[:, 31]==x], axis=0)  # Считаем сумму значение по кластеру

  # Отображаем полученную информацию по указанному кластеру
  print("Город")
  print("Россия: ", round(100*mX[0]), "%, ", sX[0], " шт", sep="") 
  print("Европа: ", round(100*mX[1]), "%, ", sX[1], " шт", sep="")
  print("Северная Америка: ", round(100*mX[2]), "%, ", sX[2], " шт", sep="") 
  print("СНГ: ", round(100*mX[3]), "%, ", sX[3], " шт", sep="")
  print("Другое: ", round(100*mX[4]), "%, ", sX[4], " шт", sep="") 

  print("\nВозраст")
  print("до 18 лет: ", round(100*mX[5]), "%, ", sX[5], " шт", sep="") 
  print("18 - 23: ", round(100*mX[6]), "%, ", sX[6], " шт", sep="")     
  print("23 - 30: ", round(100*mX[7]), "%, ", sX[7], " шт", sep="") 
  print("30 - 40: ", round(100*mX[8]), "%, ", sX[8], " шт", sep="")
  print("40 - 50: ", round(100*mX[9]), "%, ", sX[9], " шт", sep="") 
  print("50 - 60: ", round(100*mX[10]), "%, ", sX[10], " шт", sep="")
  print("60+: ", round(100*mX[11]), "%, ", sX[11], " шт", sep="")
  
  print("\nСфера работы")
  print("IT сфера (разработчик, тестировщик, администратор и т.п.): ", round(100*mX[12]), "%, ", sX[12], " шт", sep="")
  print("Профессии связанные с числами (аналитик, бухгалтер, инженер и т.п.): ", round(100*mX[13]), "%, ", sX[12], " шт", sep="")
  print("Гуманитарий (общение с людьми, искусство, медицина и т.д.): ", round(100*mX[14]), "%, ", sX[14], " шт", sep="")
  print("Предприниматель, руководитель: ", round(100*mX[15]), "%, ", sX[15], " шт", sep="")
  print("Преподаватель, учитель: ", round(100*mX[16]), "%, ", sX[16], " шт", sep="")
  print("Школьник, студент: ", round(100*mX[17]), "%, ", sX[17], " шт", sep="")
  print("Пенсионер: ", round(100*mX[18]), "%, ", sX[18], " шт", sep="")
  print("Декрет: ", round(100*mX[19]), "%, ", sX[19], " шт", sep="")

  print("\nДоход")
  print("0 руб.: ", round(100*mX[20]), "%, ", sX[20], " шт", sep="") 
  print("до 30 000 руб.: ", round(100*mX[21]), "%, ", sX[21], " шт", sep="")    
  print("до 60 000 руб.: ", round(100*mX[22]), "%, ", sX[12], " шт", sep="") 
  print("до 100 000 руб.: ", round(100*mX[23]), "%, ", sX[23], " шт", sep="") 
  print("более 100 000 руб.: ", round(100*mX[24]), "%, ", sX[24], " шт", sep="") 


  print("\nОбучение")
  print("Да, если я точно найду работу после обучения.: ", round(100*mX[25]), "%, ", sX[25], " шт", sep="")        
  print("Да, если это поможет мне в реализации проекта.: ", round(100*mX[26]), "%, ", sX[26], " шт", sep="")  
  print("Нет. Хочу получить только бесплатные материалы.: ", round(100*mX[27]), "%, ", sX[27], " шт", sep="")  

  print("\nВремя на обучение")
  print("до 5 часов в неделю: ", round(100*mX[28]), "%, ", sX[28], " шт", sep="")   
  print("до 10 часов в неделю: ", round(100*mX[29]), "%, ", sX[29], " шт", sep="") 
  print("более 10 часов в неделю: ", round(100*mX[30]), "%, ", sX[30], " шт", sep="") 

# **Шаг 1 - Получаем имя трафиколога по каждой ссылке + объединяем таблицы**

In [ ]:
os.makedirs('Отчеты csv', exist_ok=True)                      # Создание папки для данных, если её не существует
i_df = pd.read_csv(intensive)                                 # Получаем датафрейм таблицы Интенсив
a_df = pd.read_csv(anketa)                                    # Получаем датафрейм таблицы Анкета

i_df = i_df.iloc[:,:12]                                       # Оставляем только нужные для анализа столбцы
a_df = a_df.iloc[:,:11]                                       # Оставляем только нужные для анализа столбцы
a_df.drop(['roistat_url', 'requestid'], axis=1, inplace=True) # Оставляем только нужные для анализа столбцы

i_df.insert(4, 'trafficologist', 'Неизвестно')                # Добавляем столбец trafficologist для записи имени трафиколога
i_df.insert(5, 'account', 'Неизвестно 1')                       # Добавляем столбец account для записи аккаунта трафиколога

In [ ]:
# Приводим названия емейлов к нижнему регистру
# Для того, чтобы попробовать не потерять часть строк
for i in range(i_df['Email'].shape[0]):
  i_df.loc[i, 'Email'] = i_df.loc[i, 'Email'].lower()
for i in range(a_df['email'].shape[0]):
  a_df.loc[i, 'email'] = a_df.loc[i, 'email'].lower()  

In [ ]:
# Создаем словарь, где ключи - метки аккаунтов, значения - владельцы аккаунтов
trafficologist = {'facebook6': 'Юля', 'facebook3': 'Юля', 'facebook7': 'Юля', 'facebook8': 'Юля', 'facebook10': 'Юля', 'facebook14': 'Юля',
                  'facebook15': 'Ксения', 'facebook16': 'Ксения', 'facebook32': 'Ксения',
                  'facebook19': 'Евгений', 'facebook18': 'Евгений', 'facebook38': 'Евгений',
                  'facebook24': 'Михаил', 'facebook25': 'Михаил',
                  'facebook23': 'Яна', 'facebook26': 'Яна',
                  'facebook30': 'Павел',
                  'facebook22': 'Демура',
                  'facebook34': 'Руслан',
                  'facebook37': 'Валерий',
                  'facebook39': 'Артем', 'facebook40': 'Артем',
                  'facebook17': 'Алина', 'facebook29': 'Алина', 'facebook35': 'Алина', 'facebook36': 'Алина', 'facebook33': 'Алина',
                  'direct1': 'Юля Директ',
                  'direct11': 'ArrowMedia Директ',
                  'direct27': 'Мураб Директ'
                  }

In [ ]:
# Создаем словарь, где ключи - метки аккаунтов, значения - названия аккаунтов
account = {'facebook6': 'Юля 1', 'facebook3': 'Юля 2', 'facebook7': 'Юля 3', 'facebook8': 'Юля 4', 'facebook10': 'Юля 5', 'facebook14': 'Юля 7',
           'facebook15': 'Ксения 1', 'facebook16': 'Ксения 2', 'facebook32': 'Ксения 3',
           'facebook19': 'Евгений 1', 'facebook18': 'Евгений 2', 'facebook38': 'Евгений 3',
           'facebook24': 'Михаил 1', 'facebook25': 'Михаил 2',
           'facebook23': 'Яна 1', 'facebook26': 'Яна 2',
           'facebook30': 'Павел 1',
           'facebook22': 'Демура 1',
           'facebook34': 'Руслан 1',
           'facebook37': 'Валерий 1',
           'facebook39': 'Артем 1', 'facebook40': 'Артем 2',
           'facebook17': 'Алина 1', 'facebook29': 'Алина 2', 'facebook35': 'Алина 3', 'facebook36': 'Алина 4', 'facebook33': 'Алина 5',
           'direct1': 'Юля Директ',
           'direct11': 'ArrowMedia Директ',
           'direct27': 'Мураб Директ'
           }

In [ ]:
# Проходим по каждой строке и в стобцы trafficologist и account записываем
# Имена трафикологов и их аккаунты соответственно
links_list = [] # Сохраняем в список ссылки, не содержащие метки аккаунтов
for i in range(i_df.shape[0]):
  try:
    i_df.trafficologist[i] = trafficologist[i_df.roistat_url[i][i_df.roistat_url[i].find('rs=facebook')+3:i_df.roistat_url[i].find(('_'), i_df.roistat_url[i].find('rs=facebook'))]]
    i_df.account[i] = account[i_df.roistat_url[i][i_df.roistat_url[i].find('rs=facebook')+3:i_df.roistat_url[i].find(('_'), i_df.roistat_url[i].find('rs=facebook'))]]

  except KeyError:
    links_list.append(i_df.roistat_url[i])
    pass
  except AttributeError:
    links_list.append(i_df.roistat_url[i])
    pass  
  except TypeError:
    links_list.append(i_df.roistat_url[i])
    pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
i_df.head()

,Name,Email,Phone,sp_book_id,trafficologist,account,roistat_url,requestid,sent,Unnamed: 7,utm_source,utm_medium,utm_campaign,utm_content
0,Алексей,aleksei.shatskikh@gmail.com,+7 (910) 286-12-68,89258482.0,Юля,Юля 5,https://neural-university.ru/intensive_freeday...,1118292:1884310511,2021-04-22,13:41:52,InstaFb,NaN,NaN,NaN
1,Арина,arina.krkr@yandex.ru,+7 (994) 556-55-55,89251393.0,Алина,Алина 1,https://neural-university.ru/intensive_freeday...,1118292:1830365041,2021-04-08,NaN,facebook_ads,kuznetsova,Камп 3_1,1
2,Мия,miau.uu@mail.ru,+7 (965) 872-31-24,89251393.0,Яна,Яна 2,https://neural-university.ru/intensive_freeday...,1118292:1830383381,2021-04-08,NaN,PavlovaYana,NaN,NaN,NaN
3,Юлия,ablaze2@gmail.com,+39 (348) 9887-891,89251393.0,Юля,Юля 4,https://neural-university.ru/intensive_freeday...,1118292:1830396951,2021-04-08,NaN,InstaFb,NaN,NaN,NaN
4,Артем,asgpro@ya.ru,+7 (922) 517-19-90,89251393.0,Михаил,Михаил 1,https://neural-university.ru/intensive_freeday...,1118292:1830416281,2021-04-08,NaN,Konvers_RUS3,mg_30_50_in_rf,f1_t1,NaN


In [ ]:
a_df.head()

,country,Сколько_вам_лет,В_какой_сфере_сейчас_работаете,Ваш_средний_доход_в_месяц,Рассматриваете_ли_в_перспективе_платное_обучение_профессии_Разработчик_Искусственного_Интеллекта,Сколько_времени_готовы_выделить_на_обучение_в_неделю,lp_id,email,sent
0,Другое,40 - 50,"Предприниматель, руководитель",более 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 10 часов в неделю,50630,vdv-2012@bk.ru,2021-04-13
1,Россия,23 - 30,"Предприниматель, руководитель",до 30 000 руб.,Нет. Хочу получить только бесплатные материалы.,более 10 часов в неделю,50630,soujoooo@gmail.com,2021-04-13
2,Россия,40 - 50,"Профессии связанные с числами (аналитик, бухга...",до 100 000 руб.,Нет. Хочу получить только бесплатные материалы.,до 10 часов в неделю,50630,rassohinmv@gmail.com,2021-04-13
3,Россия,30 - 40,"Преподаватель, учитель",до 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 5 часов в неделю,50630,rrrnatali@mail.ru,2021-04-13
4,Другое,40 - 50,"Предприниматель, руководитель",более 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 10 часов в неделю,50630,vdv-2012@bk.ru,2021-04-13


In [ ]:
result = a_df.merge(i_df[['Email', 'trafficologist', 'account', 'roistat_url', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_content']], left_on='email', right_on='Email', how='left')
result.fillna(0, inplace=True) # Все пустые строки после объединения заполняем нулями
result.sent = pd.to_datetime(result.sent) # Переводим столбец sent в формат даты

In [ ]:
result.head()

,country,Сколько_вам_лет,В_какой_сфере_сейчас_работаете,Ваш_средний_доход_в_месяц,Рассматриваете_ли_в_перспективе_платное_обучение_профессии_Разработчик_Искусственного_Интеллекта,Сколько_времени_готовы_выделить_на_обучение_в_неделю,lp_id,email,sent,Email,trafficologist,account,roistat_url,utm_source,utm_medium,utm_campaign,utm_content
0,Другое,40 - 50,"Предприниматель, руководитель",более 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 10 часов в неделю,50630,vdv-2012@bk.ru,2021-04-13,vdv-2012@bk.ru,Михаил,Михаил 2,https://neural-university.ru/intensive_freeday...,zharkov_ra2,mir_mg25_53_job,2f_1t,0
1,Россия,23 - 30,"Предприниматель, руководитель",до 30 000 руб.,Нет. Хочу получить только бесплатные материалы.,более 10 часов в неделю,50630,soujoooo@gmail.com,2021-04-13,soujoooo@gmail.com,Алина,Алина 1,https://neural-university.ru/intensive_freeday...,kuznetsova,0,Камп 3_1,1
2,Россия,40 - 50,"Профессии связанные с числами (аналитик, бухга...",до 100 000 руб.,Нет. Хочу получить только бесплатные материалы.,до 10 часов в неделю,50630,rassohinmv@gmail.com,2021-04-13,rassohinmv@gmail.com,Евгений,Евгений 3,https://neural-university.ru/intensive_freeday...,nikulshin3,social_nik2,intensiv,w1
3,Россия,30 - 40,"Преподаватель, учитель",до 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 5 часов в неделю,50630,rrrnatali@mail.ru,2021-04-13,rrrnatali@mail.ru,Юля,Юля 2,https://neural-university.ru/intensive_freeday...,InstaFb,0,0,0
4,Другое,40 - 50,"Предприниматель, руководитель",более 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 10 часов в неделю,50630,vdv-2012@bk.ru,2021-04-13,vdv-2012@bk.ru,Михаил,Михаил 2,https://neural-university.ru/intensive_freeday...,zharkov_ra2,mir_mg25_53_job,2f_1t,0


In [ ]:
result.shape

(1241, 17)

In [ ]:
result.to_csv('Отчеты csv/trafficologist.csv', index=False)
result.to_csv(save_path + 'trafficologist.csv', index=False)
result.head(30)

,country,Сколько_вам_лет,В_какой_сфере_сейчас_работаете,Ваш_средний_доход_в_месяц,Рассматриваете_ли_в_перспективе_платное_обучение_профессии_Разработчик_Искусственного_Интеллекта,Сколько_времени_готовы_выделить_на_обучение_в_неделю,lp_id,email,sent,Email,trafficologist,account,roistat_url,utm_source,utm_medium,utm_campaign,utm_content
0,Другое,40 - 50,"Предприниматель, руководитель",более 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 10 часов в неделю,50630,vdv-2012@bk.ru,2021-04-13,vdv-2012@bk.ru,Михаил,Михаил 2,https://neural-university.ru/intensive_freeday...,zharkov_ra2,mir_mg25_53_job,2f_1t,0
1,Россия,23 - 30,"Предприниматель, руководитель",до 30 000 руб.,Нет. Хочу получить только бесплатные материалы.,более 10 часов в неделю,50630,soujoooo@gmail.com,2021-04-13,soujoooo@gmail.com,Алина,Алина 1,https://neural-university.ru/intensive_freeday...,kuznetsova,0,Камп 3_1,1
2,Россия,40 - 50,"Профессии связанные с числами (аналитик, бухга...",до 100 000 руб.,Нет. Хочу получить только бесплатные материалы.,до 10 часов в неделю,50630,rassohinmv@gmail.com,2021-04-13,rassohinmv@gmail.com,Евгений,Евгений 3,https://neural-university.ru/intensive_freeday...,nikulshin3,social_nik2,intensiv,w1
3,Россия,30 - 40,"Преподаватель, учитель",до 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 5 часов в неделю,50630,rrrnatali@mail.ru,2021-04-13,rrrnatali@mail.ru,Юля,Юля 2,https://neural-university.ru/intensive_freeday...,InstaFb,0,0,0
4,Другое,40 - 50,"Предприниматель, руководитель",более 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 10 часов в неделю,50630,vdv-2012@bk.ru,2021-04-13,vdv-2012@bk.ru,Михаил,Михаил 2,https://neural-university.ru/intensive_freeday...,zharkov_ra2,mir_mg25_53_job,2f_1t,0
5,СНГ,50 - 60,"Гуманитарий (общение с людьми, искусство, меди...",до 60 000 руб.,"Да, если я точно найду работу после обучения.",до 5 часов в неделю,50630,chernets.mariya@gmail.com,2021-04-13,chernets.mariya@gmail.com,Михаил,Михаил 2,https://neural-university.ru/intensive_freeday...,0,0,0,0
6,СНГ,50 - 60,"Гуманитарий (общение с людьми, искусство, меди...",до 60 000 руб.,"Да, если я точно найду работу после обучения.",до 5 часов в неделю,50630,chernets.mariya@gmail.com,2021-04-13,chernets.mariya@gmail.com,Михаил,Михаил 2,https://neural-university.ru/intensive_freeday...,zharkov_ra2,din3mir_mg23_55_ii_startap,7f_3t,0
7,Россия,40 - 50,"Гуманитарий (общение с людьми, искусство, меди...",более 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 10 часов в неделю,50630,x350@yandex.ru,2021-04-13,x350@yandex.ru,Ксения,Ксения 3,https://neural-university.ru/intensive_freeday...,0,cv,conv_interests,python
8,Россия,18 - 23,"Школьник, студент",0 руб.,Нет. Хочу получить только бесплатные материалы.,до 10 часов в неделю,50630,dprits@yandex.ru,2021-04-13,dprits@yandex.ru,Демура,Демура 1,https://neural-university.ru/intensive_freeday...,0,cpl,conversion,0
9,Россия,18 - 23,"Школьник, студент",до 30 000 руб.,Нет. Хочу получить только бесплатные материалы.,до 5 часов в неделю,50630,valerokun@yandex.ru,2021-04-13,valerokun@yandex.ru,Неизвестно,Неизвестно 1,https://neural-university.ru/raspisanie_kursov,0,0,0,0


**часть строк при объединении остается без ссылок, проблемы с разным написанием емейлов**

In [ ]:
# Создаем отдельную таблицу с такими строками
# А в исходной таблице эти строки удаляем
unrecognized_rows = result.copy()
unrecognized_rows = unrecognized_rows[unrecognized_rows['trafficologist'] == 0]
unrecognized_rows.to_csv('Отчеты csv/unrecognized_rows.csv', index=False)
unrecognized_rows.to_csv(save_path + 'unrecognized_rows.csv', index=False)
result = result.loc[result.trafficologist != 0]
result.index = [i for i in range(result.shape[0])]

In [ ]:
result.shape

(1140, 17)

In [ ]:
# Сохраняем отдельно емейлы из тех строк, в которых нет ссылки
skip_emails = pd.DataFrame(list(unrecognized_rows.email))
skip_emails.to_csv('Отчеты csv/skip_emails.csv', index=False)
skip_emails.to_csv(save_path + 'skip_emails.csv', index=False)

In [ ]:
# Сохраняем ссылки в которых нет идентификатора аккаунта
skip_links = pd.DataFrame(links_list)
skip_links.to_csv('Отчеты csv/skip_links.csv', index=False)
skip_links.to_csv(save_path + 'skip_links.csv', index=False)

In [ ]:
result.head(2)

,country,Сколько_вам_лет,В_какой_сфере_сейчас_работаете,Ваш_средний_доход_в_месяц,Рассматриваете_ли_в_перспективе_платное_обучение_профессии_Разработчик_Искусственного_Интеллекта,Сколько_времени_готовы_выделить_на_обучение_в_неделю,lp_id,email,sent,Email,trafficologist,account,roistat_url,utm_source,utm_medium,utm_campaign,utm_content
0,Другое,40 - 50,"Предприниматель, руководитель",более 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 10 часов в неделю,50630,vdv-2012@bk.ru,2021-04-13,vdv-2012@bk.ru,Михаил,Михаил 2,https://neural-university.ru/intensive_freeday...,zharkov_ra2,mir_mg25_53_job,2f_1t,0
1,Россия,23 - 30,"Предприниматель, руководитель",до 30 000 руб.,Нет. Хочу получить только бесплатные материалы.,более 10 часов в неделю,50630,soujoooo@gmail.com,2021-04-13,soujoooo@gmail.com,Алина,Алина 1,https://neural-university.ru/intensive_freeday...,kuznetsova,0,Камп 3_1,1


In [ ]:
result.country

0       Другое
1       Россия
2       Россия
3       Россия
4       Другое
         ...  
1135    Россия
1136    Россия
1137    Россия
1138    Россия
1139    Россия
Name: country, Length: 1140, dtype: object

In [ ]:
result_new = result.copy()

In [ ]:
result_new.insert(6, 'target_class', 0)  

In [ ]:
result_new.head()

,country,Сколько_вам_лет,В_какой_сфере_сейчас_работаете,Ваш_средний_доход_в_месяц,Рассматриваете_ли_в_перспективе_платное_обучение_профессии_Разработчик_Искусственного_Интеллекта,Сколько_времени_готовы_выделить_на_обучение_в_неделю,target_class,lp_id,email,sent,Email,trafficologist,account,roistat_url,utm_source,utm_medium,utm_campaign,utm_content
0,Другое,40 - 50,"Предприниматель, руководитель",более 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 10 часов в неделю,5,50630,vdv-2012@bk.ru,2021-04-13,vdv-2012@bk.ru,Михаил,Михаил 2,https://neural-university.ru/intensive_freeday...,zharkov_ra2,mir_mg25_53_job,2f_1t,0
1,Россия,23 - 30,"Предприниматель, руководитель",до 30 000 руб.,Нет. Хочу получить только бесплатные материалы.,более 10 часов в неделю,4,50630,soujoooo@gmail.com,2021-04-13,soujoooo@gmail.com,Алина,Алина 1,https://neural-university.ru/intensive_freeday...,kuznetsova,0,Камп 3_1,1
2,Россия,40 - 50,"Профессии связанные с числами (аналитик, бухга...",до 100 000 руб.,Нет. Хочу получить только бесплатные материалы.,до 10 часов в неделю,5,50630,rassohinmv@gmail.com,2021-04-13,rassohinmv@gmail.com,Евгений,Евгений 3,https://neural-university.ru/intensive_freeday...,nikulshin3,social_nik2,intensiv,w1
3,Россия,30 - 40,"Преподаватель, учитель",до 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 5 часов в неделю,5,50630,rrrnatali@mail.ru,2021-04-13,rrrnatali@mail.ru,Юля,Юля 2,https://neural-university.ru/intensive_freeday...,InstaFb,0,0,0
4,Другое,40 - 50,"Предприниматель, руководитель",более 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 10 часов в неделю,5,50630,vdv-2012@bk.ru,2021-04-13,vdv-2012@bk.ru,Михаил,Михаил 2,https://neural-university.ru/intensive_freeday...,zharkov_ra2,mir_mg25_53_job,2f_1t,0


In [ ]:
for i in range(result_new.shape[0]):
  target_class = 0
  if result_new.loc[i, 'country'] in countries[:3]:
    target_class += 1
  if result_new.loc[i, 'Сколько_вам_лет'] in ages[2:6]:
    target_class += 1
  if result_new.loc[i, 'В_какой_сфере_сейчас_работаете'] in jobs[0:5]:
    target_class += 1
  if result_new.loc[i, 'Ваш_средний_доход_в_месяц'] in earnings[2:]:
    target_class += 1         
  if result_new.loc[i, 'Рассматриваете_ли_в_перспективе_платное_обучение_профессии_Разработчик_Искусственного_Интеллекта'] in trainings[:2]:
    target_class += 1
  if result_new.loc[i, 'Сколько_времени_готовы_выделить_на_обучение_в_неделю'] in times[1:]:
    target_class += 1

  result_new.loc[i, 'target_class'] = target_class


In [ ]:
result_new[result_new.target_class == 0].shape

(3, 18)

In [ ]:
# Список категорий по приоритетности
countries = ['Россия', 'Европа', 'Северная Америка', 'СНГ', 'Другое']
ages = ['до 18 лет', '18 - 23', '23 - 30', '30 - 40', '40 - 50', '50 - 60', '60 +']
jobs = ['IT сфера (разработчик, тестировщик, администратор и т.п.)', 'Профессии связанные с числами (аналитик, бухгалтер, инженер и т.п.)', 'Гуманитарий (общение с людьми, искусство, медицина и т.д.)',
       'Предприниматель, руководитель',
       'Преподаватель, учитель', 'Школьник, студент', 'Пенсионер', 'Декрет', ]
earnings = ['0 руб.', 'до 30 000 руб.', 'до 60 000 руб.', 'до 100 000 руб.', 'более 100 000 руб.']
trainings = ['Да, если я точно найду работу после обучения.', 'Да, если это поможет мне в реализации проекта.', 'Нет. Хочу получить только бесплатные материалы.']
times = ['до 5 часов в неделю', 'до 10 часов в неделю', 'более 10 часов в неделю']

# **Шаг 2 - Сегментация**

**В переменные start_date и end_time записываем дату в формате '2021-04-14'**

start_date - дата начала фильтрации

end_date - дата конца фильтрации

Если нужно фильтровать с самого начала, start_date = '0'

Если нужно фильтровать с до самого конца, end_date = '0'

**То есть, если фильтрация не нужна, записываем start_date = '0', end_date = '0'**

In [ ]:
# Задаем даты
start_date = '0'
end_date = '0'

In [ ]:
# Фильтруем датасет
filtered_result = filter_df(start_date, end_date)

## Предварительные расчеты

In [ ]:
filtered_result.shape

(1140, 17)

In [ ]:
result.shape

(1140, 17)

In [ ]:
# Список категорий по приоритетности
countries = ['Россия', 'Европа', 'Северная Америка', 'СНГ', 'Другое']
ages = ['до 18 лет', '18 - 23', '23 - 30', '30 - 40', '40 - 50', '50 - 60', '60 +']
jobs = ['IT сфера (разработчик, тестировщик, администратор и т.п.)', 'Профессии связанные с числами (аналитик, бухгалтер, инженер и т.п.)', 'Гуманитарий (общение с людьми, искусство, медицина и т.д.)',
       'Предприниматель, руководитель',
       'Преподаватель, учитель', 'Школьник, студент', 'Пенсионер', 'Декрет', ]
earnings = ['0 руб.', 'до 30 000 руб.', 'до 60 000 руб.', 'до 100 000 руб.', 'более 100 000 руб.']
trainings = ['Да, если я точно найду работу после обучения.', 'Да, если это поможет мне в реализации проекта.', 'Нет. Хочу получить только бесплатные материалы.']
times = ['до 5 часов в неделю', 'до 10 часов в неделю', 'более 10 часов в неделю']

In [ ]:
# Список названий колонок
list(result.columns.values[:6])

['country',
 'Сколько_вам_лет',
 'В_какой_сфере_сейчас_работаете',
 'Ваш_средний_доход_в_месяц',
 'Рассматриваете_ли_в_перспективе_платное_обучение_профессии_Разработчик_Искусственного_Интеллекта',
 'Сколько_времени_готовы_выделить_на_обучение_в_неделю']

In [ ]:
# Объединяем массиввы уникальных значений в колонках trafficologist и account
# Оставляем только уникальные значения, переводи массив в список и сортируем
created_columns = np.unique(np.concatenate((filtered_result['trafficologist'].unique(), filtered_result['account'].unique()))).tolist()
created_columns.sort()
created_columns

['Алина',
 'Алина 1',
 'Алина 2',
 'Алина 3',
 'Алина 4',
 'Алина 5',
 'Артем',
 'Артем 1',
 'Артем 2',
 'Валерий',
 'Валерий 1',
 'Демура',
 'Демура 1',
 'Евгений',
 'Евгений 1',
 'Евгений 3',
 'Ксения',
 'Ксения 1',
 'Ксения 2',
 'Ксения 3',
 'Михаил',
 'Михаил 1',
 'Михаил 2',
 'Неизвестно',
 'Неизвестно 1',
 'Павел',
 'Павел 1',
 'Юля',
 'Юля 1',
 'Юля 2',
 'Юля 4',
 'Юля 5',
 'Яна',
 'Яна 1',
 'Яна 2']

In [ ]:
delete_columns = ['Валерий 1', 'Демура 1', 'Неизвестно 1', 'Павел 1']

## **Страны**

In [ ]:
df_countries, df_val_countries = channels_table(df = filtered_result, category = countries, column_name = 'country')

In [ ]:
df_countries.to_csv('Отчеты csv/df_countries.csv', index=False)
df_countries.to_csv(save_path + 'df_countries.csv', index=False)
df_countries

,country,Все,Алина,Алина 1,Алина 2,Алина 3,Алина 4,Алина 5,Артем,Артем 1,Артем 2,Валерий,Демура,Евгений,Евгений 1,Евгений 3,Ксения,Ксения 1,Ксения 2,Ксения 3,Михаил,Михаил 1,Михаил 2,Неизвестно,Павел,Юля,Юля 1,Юля 2,Юля 4,Юля 5,Яна,Яна 1,Яна 2
0,Россия,72.80,98,100,100,100,92,99,100,100,100,100,93,98,97,100,100,100,100,100,41,96,10,88,100,45,8,100,10,100,64,51,100
1,Европа,13.34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,24,4,0,42,71,0,70,0,22,31,0
2,Северная Америка,1.93,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,4,0,4,6,0,7,0,5,6,0
3,СНГ,6.50,2,0,0,0,8,1,0,0,0,0,7,2,3,0,0,0,0,0,20,2,29,4,0,7,12,0,10,0,2,3,0
4,Другое,5.43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,1,35,0,0,2,3,0,3,0,7,9,0
5,ЦА,88.07,98,100,100,100,92,99,100,100,100,100,93,98,97,100,100,100,100,100,57,96,36,96,100,91,85,100,87,100,91,88,100


In [ ]:
df_val_countries.to_csv('Отчеты csv/df_val_countries.csv', index=False)
df_val_countries.to_csv(save_path + 'df_val_countries.csv', index=False)
df_val_countries

,country,Все,Алина,Алина 1,Алина 2,Алина 3,Алина 4,Алина 5,Артем,Артем 1,Артем 2,Валерий,Демура,Евгений,Евгений 1,Евгений 3,Ксения,Ксения 1,Ксения 2,Ксения 3,Михаил,Михаил 1,Михаил 2,Неизвестно,Павел,Юля,Юля 1,Юля 2,Юля 4,Юля 5,Яна,Яна 1,Яна 2
0,Россия,830,187,56,1,14,37,79,8,5,3,21,28,65,36,29,108,38,23,47,93,78,15,140,17,94,5,60,6,23,69,40,29
1,Европа,152,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34,0,34,6,0,88,46,0,42,0,24,24,0
2,Северная Америка,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,3,6,0,8,4,0,4,0,5,5,0
3,СНГ,74,4,0,0,0,3,1,0,0,0,0,2,1,1,0,0,0,0,0,44,2,42,7,0,14,8,0,6,0,2,2,0
4,Другое,62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51,1,50,0,0,4,2,0,2,0,7,7,0
5,ЦА,1004,187,56,1,14,37,79,8,5,3,21,28,65,36,29,108,38,23,47,130,78,52,152,17,190,55,60,52,23,98,69,29


##**Возраст**

In [ ]:
df_ages, df_ages_val = channels_table(df = filtered_result, category = ages, column_name = 'Сколько_вам_лет')

In [ ]:
df_ages.to_csv('Отчеты csv/df_ages.csv', index=False)
df_ages.to_csv(save_path + 'df_ages.csv', index=False)
df_ages

,Сколько_вам_лет,Все,Алина,Алина 1,Алина 2,Алина 3,Алина 4,Алина 5,Артем,Артем 1,Артем 2,Валерий,Демура,Евгений,Евгений 1,Евгений 3,Ксения,Ксения 1,Ксения 2,Ксения 3,Михаил,Михаил 1,Михаил 2,Неизвестно,Павел,Юля,Юля 1,Юля 2,Юля 4,Юля 5,Яна,Яна 1,Яна 2
0,до 18 лет,4.31,3,4,0,0,2,2,0,0,0,29,7,6,5,7,2,3,0,2,3,1,4,6,6,4,5,2,3,9,5,6,0
1,18 - 23,9.04,8,12,0,0,5,9,0,0,0,52,17,5,8,0,9,8,13,9,9,11,8,12,59,3,5,3,2,0,2,3,0
2,23 - 30,17.02,27,27,0,36,30,25,25,20,33,0,47,17,22,10,18,18,22,15,11,17,8,13,12,14,17,8,15,22,17,14,24
3,30 - 40,35.69,43,46,0,29,40,45,0,0,0,19,23,30,27,34,44,42,35,51,31,30,31,28,18,41,40,53,27,48,41,44,34
4,40 - 50,27.02,16,11,100,36,18,15,50,60,33,0,7,30,32,28,24,26,26,21,29,28,29,33,0,37,34,32,50,22,30,27,38
5,50 - 60,6.06,3,0,0,0,5,4,25,20,33,0,0,12,5,21,2,0,4,2,14,11,16,7,0,1,0,2,3,0,6,6,3
6,60 +,0.89,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,3,1,3,1,6,0,0,0,0,0,0,0,0
7,ЦА,85.79,89,84,100,101,93,89,100,100,99,19,77,89,86,93,88,86,87,89,85,86,84,81,30,93,91,95,95,92,94,91,99


In [ ]:
df_ages_val.to_csv('Отчеты csv/df_ages_val.csv', index=False)
df_ages_val.to_csv(save_path + 'df_ages_val.csv', index=False)
df_ages_val

,Сколько_вам_лет,Все,Алина,Алина 1,Алина 2,Алина 3,Алина 4,Алина 5,Артем,Артем 1,Артем 2,Валерий,Демура,Евгений,Евгений 1,Евгений 3,Ксения,Ксения 1,Ксения 2,Ксения 3,Михаил,Михаил 1,Михаил 2,Неизвестно,Павел,Юля,Юля 1,Юля 2,Юля 4,Юля 5,Яна,Яна 1,Яна 2
0,до 18 лет,49,5,2,0,0,1,2,0,0,0,6,2,4,2,2,2,1,0,1,7,1,6,9,1,8,3,1,2,2,5,5,0
1,18 - 23,103,16,7,0,0,2,7,0,0,0,11,5,3,3,0,10,3,3,4,21,9,12,19,10,6,3,2,1,0,2,2,0
2,23 - 30,194,52,15,0,5,12,20,2,1,1,0,14,11,8,3,19,7,5,7,25,14,11,21,2,30,11,5,9,5,18,11,7
3,30 - 40,407,82,26,0,4,16,36,0,0,0,4,7,20,10,10,48,16,8,24,69,24,45,45,3,85,26,32,16,11,44,34,10
4,40 - 50,308,31,6,1,5,7,12,4,3,1,0,2,20,12,8,26,10,6,10,65,23,42,52,0,76,22,19,30,5,32,21,11
5,50 - 60,69,5,0,0,0,2,3,2,1,1,0,0,8,2,6,2,0,1,1,32,9,23,11,0,3,0,1,2,0,6,5,1
6,60 +,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,6,1,5,2,1,0,0,0,0,0,0,0,0
7,ЦА,978,170,47,1,14,37,71,8,5,3,4,23,59,32,27,95,33,20,42,191,70,121,129,5,194,59,57,57,21,100,71,29


##**Профессия**

In [ ]:
df_jobs, df_jobs_val = channels_table(df = filtered_result, category = jobs, column_name = 'В_какой_сфере_сейчас_работаете')

In [ ]:
df_jobs.to_csv('Отчеты csv/df_jobs.csv', index=False)
df_jobs.to_csv(save_path + 'df_jobs.csv', index=False)
df_jobs

,В_какой_сфере_сейчас_работаете,Все,Алина,Алина 1,Алина 2,Алина 3,Алина 4,Алина 5,Артем,Артем 1,Артем 2,Валерий,Демура,Евгений,Евгений 1,Евгений 3,Ксения,Ксения 1,Ксения 2,Ксения 3,Михаил,Михаил 1,Михаил 2,Неизвестно,Павел,Юля,Юля 1,Юля 2,Юля 4,Юля 5,Яна,Яна 1,Яна 2
0,"IT сфера (разработчик, тестировщик, администра...",26.23,38,39,100,36,42,34,12,0,33,5,23,24,32,14,31,32,39,26,25,25,25,20,12,33,25,48,28,30,9,12,3
1,"Профессии связанные с числами (аналитик, бухга...",18.23,23,20,0,36,22,24,38,40,33,14,23,17,11,24,18,13,9,26,18,28,12,19,12,14,20,7,10,26,19,14,31
2,"Гуманитарий (общение с людьми, искусство, меди...",17.02,9,7,0,0,8,14,12,20,0,0,7,21,24,17,19,18,22,17,17,15,19,17,18,16,18,7,23,13,35,35,34
3,"Предприниматель, руководитель",13.95,13,12,0,14,10,14,12,0,33,0,3,20,16,24,15,16,22,11,19,19,19,11,0,17,14,18,18,22,7,6,10
4,"Преподаватель, учитель",5.36,6,7,0,14,10,2,12,20,0,5,7,3,5,0,6,11,4,4,3,4,2,8,0,6,0,13,7,0,5,4,7
5,"Школьник, студент",12.64,8,11,0,0,2,10,0,0,0,71,27,8,5,10,9,8,4,13,11,6,14,19,53,8,14,2,8,9,9,13,0
6,Пенсионер,3.07,2,0,0,0,5,1,12,20,0,0,0,8,5,10,1,0,0,2,4,1,6,4,6,1,2,3,0,0,5,3,10
7,Декрет,3.50,2,4,0,0,0,1,0,0,0,5,10,0,0,0,2,3,0,2,3,2,3,2,0,4,8,2,5,0,11,14,3
8,ЦА,75.43,83,78,100,86,82,86,74,60,99,19,56,82,83,79,83,79,92,80,79,87,75,67,42,80,77,80,79,91,70,67,78


In [ ]:
df_jobs_val.to_csv('Отчеты csv/df_jobs_val.csv', index=False)
df_jobs_val.to_csv(save_path + 'df_jobs_val.csv', index=False)
df_jobs_val

,В_какой_сфере_сейчас_работаете,Все,Алина,Алина 1,Алина 2,Алина 3,Алина 4,Алина 5,Артем,Артем 1,Артем 2,Валерий,Демура,Евгений,Евгений 1,Евгений 3,Ксения,Ксения 1,Ксения 2,Ксения 3,Михаил,Михаил 1,Михаил 2,Неизвестно,Павел,Юля,Юля 1,Юля 2,Юля 4,Юля 5,Яна,Яна 1,Яна 2
0,"IT сфера (разработчик, тестировщик, администра...",299,72,22,1,5,17,27,1,0,1,1,7,16,12,4,33,12,9,12,56,20,36,32,2,69,16,29,17,7,10,9,1
1,"Профессии связанные с числами (аналитик, бухга...",208,44,11,0,5,9,19,3,2,1,3,7,11,4,7,19,5,2,12,40,23,17,30,2,29,13,4,6,6,20,11,9
2,"Гуманитарий (общение с людьми, искусство, меди...",194,18,4,0,0,3,11,1,1,0,0,2,14,9,5,20,7,5,8,39,12,27,27,3,33,12,4,14,3,37,27,10
3,"Предприниматель, руководитель",159,24,7,0,2,4,11,1,0,1,0,1,13,6,7,16,6,5,5,43,15,28,17,0,36,9,11,11,5,8,5,3
4,"Преподаватель, учитель",61,12,4,0,2,4,2,1,1,0,1,2,2,2,0,7,4,1,2,6,3,3,13,0,12,0,8,4,0,5,3,2
5,"Школьник, студент",144,15,6,0,0,1,8,0,0,0,15,8,5,2,3,10,3,1,6,25,5,20,30,9,17,9,1,5,2,10,10,0
6,Пенсионер,35,3,0,0,0,2,1,1,1,0,0,0,5,2,3,1,0,0,1,9,1,8,7,1,3,1,2,0,0,5,2,3
7,Декрет,40,3,2,0,0,0,1,0,0,0,1,3,0,0,0,2,1,0,1,7,2,5,3,0,9,5,1,3,0,12,11,1
8,ЦА,860,158,44,1,12,33,68,6,3,3,4,17,54,31,23,88,30,21,37,178,70,108,106,7,167,50,48,48,21,75,52,23


##**Доход**

In [ ]:
df_earnings, df_earnings_val = channels_table(df = filtered_result, category = earnings, column_name = 'Ваш_средний_доход_в_месяц')

In [ ]:
df_earnings.to_csv('Отчеты csv/df_earnings.csv', index=False)
df_earnings.to_csv(save_path + 'df_earnings.csv', index=False)
df_earnings

,Ваш_средний_доход_в_месяц,Все,Алина,Алина 1,Алина 2,Алина 3,Алина 4,Алина 5,Артем,Артем 1,Артем 2,Валерий,Демура,Евгений,Евгений 1,Евгений 3,Ксения,Ксения 1,Ксения 2,Ксения 3,Михаил,Михаил 1,Михаил 2,Неизвестно,Павел,Юля,Юля 1,Юля 2,Юля 4,Юля 5,Яна,Яна 1,Яна 2
0,0 руб.,16.51,15,18,0,7,15,14,0,0,0,67,17,14,5,24,13,16,9,13,16,10,19,17,35,14,20,10,15,9,18,18,17
1,до 30 000 руб.,22.82,21,18,100,21,22,22,25,40,0,24,33,24,27,21,22,26,26,17,19,23,16,31,35,14,11,13,18,13,34,26,55
2,до 60 000 руб.,25.45,25,16,0,43,28,26,62,60,67,5,27,29,35,21,24,24,9,32,30,32,28,28,12,20,26,13,18,26,26,29,17
3,до 100 000 руб.,14.91,18,18,0,14,18,20,0,0,0,0,20,20,19,21,19,11,22,23,11,15,9,9,6,21,17,27,20,17,11,13,7
4,более 100 000 руб.,20.36,21,30,0,14,18,18,12,0,33,5,3,14,14,14,22,24,35,15,24,20,27,14,12,31,26,37,28,35,11,14,3
5,ЦА,60.72,64,64,0,71,64,64,74,60,100,10,50,63,68,56,65,59,66,70,65,67,64,51,30,72,69,77,66,78,48,56,27


In [ ]:
df_earnings_val.to_csv('Отчеты csv/df_earnings_val.csv', index=False)
df_earnings_val.to_csv(save_path + 'df_earnings_val.csv', index=False)
df_earnings_val

,Ваш_средний_доход_в_месяц,Все,Алина,Алина 1,Алина 2,Алина 3,Алина 4,Алина 5,Артем,Артем 1,Артем 2,Валерий,Демура,Евгений,Евгений 1,Евгений 3,Ксения,Ксения 1,Ксения 2,Ксения 3,Михаил,Михаил 1,Михаил 2,Неизвестно,Павел,Юля,Юля 1,Юля 2,Юля 4,Юля 5,Яна,Яна 1,Яна 2
0,0 руб.,188,28,10,0,1,6,11,0,0,0,14,5,9,2,7,14,6,2,6,36,8,28,27,6,30,13,6,9,2,19,14,5
1,до 30 000 руб.,260,41,10,1,3,9,18,2,2,0,5,10,16,10,6,24,10,6,8,42,19,23,49,6,29,7,8,11,3,36,20,16
2,до 60 000 руб.,290,47,9,0,6,11,21,5,3,2,1,8,19,13,6,26,9,2,15,67,26,41,45,2,42,17,8,11,6,28,23,5
3,до 100 000 руб.,170,35,10,0,2,7,16,0,0,0,0,6,13,7,6,20,4,5,11,25,12,13,15,1,43,11,16,12,4,12,10,2
4,более 100 000 руб.,232,40,17,0,2,7,14,1,0,1,1,1,9,5,4,24,9,8,7,55,16,39,23,2,64,17,22,17,8,12,11,1
5,ЦА,692,122,36,0,10,25,51,6,3,3,2,15,41,25,16,70,22,15,33,147,54,93,83,5,149,45,46,40,18,52,44,8


##**Обучение**

In [ ]:
df_trainings, df_trainings_val = channels_table(df = filtered_result, category = trainings, column_name = 'Рассматриваете_ли_в_перспективе_платное_обучение_профессии_Разработчик_Искусственного_Интеллекта')

In [ ]:
df_trainings.to_csv('Отчеты csv/df_trainings.csv', index=False)
df_trainings.to_csv(save_path + 'df_trainings.csv', index=False)
df_trainings

,Рассматриваете_ли_в_перспективе_платное_обучение_профессии_Разработчик_Искусственного_Интеллекта,Все,Алина,Алина 1,Алина 2,Алина 3,Алина 4,Алина 5,Артем,Артем 1,Артем 2,Валерий,Демура,Евгений,Евгений 1,Евгений 3,Ксения,Ксения 1,Ксения 2,Ксения 3,Михаил,Михаил 1,Михаил 2,Неизвестно,Павел,Юля,Юля 1,Юля 2,Юля 4,Юля 5,Яна,Яна 1,Яна 2
0,"Да, если я точно найду работу после обучения.",28.95,23,20,0,36,30,20,62,80,33,29,47,33,30,38,23,21,17,28,30,25,33,36,47,26,26,27,28,22,25,27,21
1,"Да, если это поможет мне в реализации проекта.",33.78,42,50,0,21,42,41,25,0,67,24,13,21,19,24,35,34,39,34,34,32,35,26,18,44,43,43,48,39,26,27,24
2,Нет. Хочу получить только бесплатные материалы.,37.28,35,30,100,43,28,39,12,20,0,48,40,45,51,38,42,45,43,38,36,43,33,38,35,29,31,30,23,39,49,46,55
3,ЦА,62.73,65,70,0,57,72,61,87,80,100,53,60,54,49,62,58,55,56,62,64,57,68,62,65,70,69,70,76,61,51,54,45


In [ ]:
df_trainings_val.to_csv('Отчеты csv/df_trainings_val.csv', index=False)
df_trainings_val.to_csv(save_path + 'df_trainings_val.csv', index=False)
df_trainings_val

,Рассматриваете_ли_в_перспективе_платное_обучение_профессии_Разработчик_Искусственного_Интеллекта,Все,Алина,Алина 1,Алина 2,Алина 3,Алина 4,Алина 5,Артем,Артем 1,Артем 2,Валерий,Демура,Евгений,Евгений 1,Евгений 3,Ксения,Ксения 1,Ксения 2,Ксения 3,Михаил,Михаил 1,Михаил 2,Неизвестно,Павел,Юля,Юля 1,Юля 2,Юля 4,Юля 5,Яна,Яна 1,Яна 2
0,"Да, если я точно найду работу после обучения.",330,44,11,0,5,12,16,5,4,1,6,14,22,11,11,25,8,4,13,67,20,47,57,8,55,17,16,17,5,27,21,6
1,"Да, если это поможет мне в реализации проекта.",385,81,28,0,3,17,33,2,0,2,5,4,14,7,7,38,13,9,16,76,26,50,42,3,92,28,26,29,9,28,21,7
2,Нет. Хочу получить только бесплатные материалы.,425,66,17,1,6,11,31,1,1,0,10,12,30,19,11,45,17,10,18,82,35,47,60,6,61,20,18,14,9,52,36,16
3,ЦА,715,125,39,0,8,29,49,7,4,3,11,18,36,18,18,63,21,13,29,143,46,97,99,11,147,45,42,46,14,55,42,13


##**Время**

In [ ]:
df_times, df_times_val = channels_table(df = filtered_result, category = times, column_name = 'Сколько_времени_готовы_выделить_на_обучение_в_неделю')

In [ ]:
df_times.to_csv('Отчеты csv/df_times.csv', index=False)
df_times.to_csv(save_path + 'df_times.csv', index=False)
df_times

,Сколько_времени_готовы_выделить_на_обучение_в_неделю,Все,Алина,Алина 1,Алина 2,Алина 3,Алина 4,Алина 5,Артем,Артем 1,Артем 2,Валерий,Демура,Евгений,Евгений 1,Евгений 3,Ксения,Ксения 1,Ксения 2,Ксения 3,Михаил,Михаил 1,Михаил 2,Неизвестно,Павел,Юля,Юля 1,Юля 2,Юля 4,Юля 5,Яна,Яна 1,Яна 2
0,до 5 часов в неделю,47.28,45,52,100,36,50,39,50,20,100,48,33,52,51,52,40,39,43,38,50,60,44,41,12,55,57,53,48,70,54,50,66
1,до 10 часов в неделю,32.90,37,27,0,43,40,42,50,80,0,33,33,30,35,24,42,34,39,49,32,32,31,32,35,28,29,30,32,9,30,31,28
2,более 10 часов в неделю,19.82,18,21,0,21,10,19,0,0,0,19,33,18,14,24,19,26,17,13,18,7,24,27,53,17,14,17,20,22,16,19,7
3,ЦА,52.72,55,48,0,64,50,61,50,80,0,52,66,48,49,48,61,60,56,62,50,39,55,59,88,45,43,47,52,31,46,50,35


In [ ]:
df_times_val.to_csv('Отчеты csv/df_times_val.csv', index=False)
df_times_val.to_csv(save_path + 'df_times_val.csv', index=False)
df_times_val

,Сколько_времени_готовы_выделить_на_обучение_в_неделю,Все,Алина,Алина 1,Алина 2,Алина 3,Алина 4,Алина 5,Артем,Артем 1,Артем 2,Валерий,Демура,Евгений,Евгений 1,Евгений 3,Ксения,Ксения 1,Ксения 2,Ксения 3,Михаил,Михаил 1,Михаил 2,Неизвестно,Павел,Юля,Юля 1,Юля 2,Юля 4,Юля 5,Яна,Яна 1,Яна 2
0,до 5 часов в неделю,539,86,29,1,5,20,31,4,1,3,10,10,34,19,15,43,15,10,18,113,49,64,65,2,114,37,32,29,16,58,39,19
1,до 10 часов в неделю,375,71,15,0,6,16,34,4,4,0,7,10,20,13,7,45,13,9,23,71,26,45,51,6,58,19,18,19,2,32,24,8
2,более 10 часов в неделю,226,34,12,0,3,4,15,0,0,0,4,10,12,5,7,20,10,4,6,41,6,35,43,9,36,9,10,12,5,17,15,2
3,ЦА,601,105,27,0,9,20,49,4,4,0,11,20,32,18,14,65,23,13,29,112,32,80,94,15,94,28,28,31,7,49,39,10


## **ЦА**

In [ ]:
df_target_audience, df_target_audience_val = target_audience_channels_table(filtered_result)

In [ ]:
df_target_audience.to_csv('Отчеты csv/df_target_audience.csv', index=False)
df_target_audience.to_csv(save_path + 'df_target_audience.csv', index=False)
df_target_audience

,Все,Алина,Алина 1,Алина 2,Алина 3,Алина 4,Алина 5,Артем,Артем 1,Артем 2,Валерий,Демура,Евгений,Евгений 1,Евгений 3,Ксения,Ксения 1,Ксения 2,Ксения 3,Михаил,Михаил 1,Михаил 2,Неизвестно,Павел,Юля,Юля 1,Юля 2,Юля 4,Юля 5,Яна,Яна 1,Яна 2
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0
1,3,1,0,0,0,0,1,0,0,0,19,3,2,0,3,2,5,0,0,5,4,6,4,6,2,5,0,2,4,1,1,0
2,9,5,9,0,0,0,5,12,20,0,43,17,2,3,0,7,3,9,11,8,6,10,10,24,4,8,0,3,4,14,13,17
3,15,12,16,100,0,12,11,0,0,0,24,13,18,22,14,10,13,9,9,17,10,21,18,29,10,12,8,10,9,14,13,17
4,26,25,20,0,50,35,21,12,20,0,5,23,29,27,31,22,26,22,19,29,32,28,18,6,27,17,40,25,26,33,33,34
5,33,35,32,0,21,40,39,38,0,100,0,27,40,38,41,33,32,39,32,26,31,24,32,24,36,42,27,42,30,29,31,24
6,18,21,23,0,29,12,22,38,60,0,10,17,11,11,10,25,21,22,30,13,17,10,18,12,21,17,25,18,26,8,9,7
7,50,56,55,0,50,52,61,75,60,100,10,43,50,49,52,59,53,61,62,39,48,35,50,35,57,59,52,60,57,37,40,31
8,76,82,75,0,100,88,82,88,80,100,14,67,79,76,83,81,79,83,81,68,80,62,68,41,84,75,92,85,83,71,73,66


In [ ]:
df_target_audience_val.to_csv('Отчеты csv/df_target_audience_val.csv', index=False)
df_target_audience_val.to_csv(save_path + 'df_target_audience_val.csv', index=False)
df_target_audience_val

,Все,Алина,Алина 1,Алина 2,Алина 3,Алина 4,Алина 5,Артем,Артем 1,Артем 2,Валерий,Демура,Евгений,Евгений 1,Евгений 3,Ксения,Ксения 1,Ксения 2,Ксения 3,Михаил,Михаил 1,Михаил 2,Неизвестно,Павел,Юля,Юля 1,Юля 2,Юля 4,Юля 5,Яна,Яна 1,Яна 2
0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2,1,0,0,0,0,0,0,0,0,0
1,33,1,0,0,0,0,1,0,0,0,4,1,1,0,1,2,2,0,0,11,3,8,6,1,5,3,0,1,1,1,1,0
2,95,9,5,0,0,0,4,1,1,0,9,5,1,1,0,8,1,2,5,19,5,14,16,4,8,5,0,2,1,15,10,5
3,163,24,9,1,0,5,9,0,0,0,5,4,12,8,4,11,5,2,4,38,8,30,28,5,21,8,5,6,2,15,10,5
4,288,49,11,0,7,14,17,1,1,0,1,7,19,10,9,24,10,5,9,66,26,40,28,1,56,11,24,15,6,36,26,10
5,362,68,18,0,3,16,31,3,0,3,0,8,26,14,12,36,12,9,15,60,25,35,51,4,75,27,16,25,7,31,24,7
6,196,40,13,0,4,5,18,3,3,0,2,5,7,4,3,27,8,5,14,29,14,15,29,2,43,11,15,11,6,9,7,2
7,558,108,31,0,7,21,49,6,3,3,2,13,33,18,15,63,20,14,29,89,39,50,80,6,118,38,31,36,13,40,31,9
8,846,157,42,0,14,35,66,7,4,3,3,20,52,28,24,87,30,19,38,155,65,90,108,7,174,49,55,51,19,76,57,19


In [ ]:
print(result_new[result_new.target_class == 0].shape[0])
print(result_new[result_new.target_class == 1].shape[0])
print(result_new[result_new.target_class == 2].shape[0])
print(result_new[result_new.target_class == 3].shape[0])
print(result_new[result_new.target_class == 4].shape[0])
print(result_new[result_new.target_class == 5].shape[0])
print(result_new[result_new.target_class == 6].shape[0])
print((result_new[(result_new.target_class == 5) | (result_new.target_class == 6)].shape[0]))
print((result_new[(result_new.target_class == 4) | (result_new.target_class == 5) | (result_new.target_class == 6)].shape[0]))

3
32
88
150
286
377
204
581
867


In [ ]:
filtered_result.shape

(1140, 17)

In [ ]:
filtered_result.head()

,country,Сколько_вам_лет,В_какой_сфере_сейчас_работаете,Ваш_средний_доход_в_месяц,Рассматриваете_ли_в_перспективе_платное_обучение_профессии_Разработчик_Искусственного_Интеллекта,Сколько_времени_готовы_выделить_на_обучение_в_неделю,lp_id,email,sent,Email,trafficologist,account,roistat_url,utm_source,utm_medium,utm_campaign,utm_content
0,Другое,40 - 50,"Предприниматель, руководитель",более 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 10 часов в неделю,50630,vdv-2012@bk.ru,2021-04-13,vdv-2012@bk.ru,Михаил,Михаил 2,https://neural-university.ru/intensive_freeday...,zharkov_ra2,mir_mg25_53_job,2f_1t,0
1,Россия,23 - 30,"Предприниматель, руководитель",до 30 000 руб.,Нет. Хочу получить только бесплатные материалы.,более 10 часов в неделю,50630,soujoooo@gmail.com,2021-04-13,soujoooo@gmail.com,Алина,Алина 1,https://neural-university.ru/intensive_freeday...,kuznetsova,0,Камп 3_1,1
2,Россия,40 - 50,"Профессии связанные с числами (аналитик, бухга...",до 100 000 руб.,Нет. Хочу получить только бесплатные материалы.,до 10 часов в неделю,50630,rassohinmv@gmail.com,2021-04-13,rassohinmv@gmail.com,Евгений,Евгений 3,https://neural-university.ru/intensive_freeday...,nikulshin3,social_nik2,intensiv,w1
3,Россия,30 - 40,"Преподаватель, учитель",до 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 5 часов в неделю,50630,rrrnatali@mail.ru,2021-04-13,rrrnatali@mail.ru,Юля,Юля 2,https://neural-university.ru/intensive_freeday...,InstaFb,0,0,0
4,Другое,40 - 50,"Предприниматель, руководитель",более 100 000 руб.,"Да, если это поможет мне в реализации проекта.",до 10 часов в неделю,50630,vdv-2012@bk.ru,2021-04-13,vdv-2012@bk.ru,Михаил,Михаил 2,https://neural-university.ru/intensive_freeday...,zharkov_ra2,mir_mg25_53_job,2f_1t,0


# **Шаг 3 - Информация по кластерам**

In [ ]:
xTrain01 = get01Data(filtered_result.values) # Создаем обучающую выборку по данным из базы
xTrain01 = addLabels(xTrain01)               # Добавляем метки кластера

In [ ]:
printCluster(6)

Размер кластера: 12
Город
Россия: 92%, 11 шт
Европа: 8%, 1 шт
Северная Америка: 0%, 0 шт
СНГ: 0%, 0 шт
Другое: 0%, 0 шт

Возраст
до 18 лет: 0%, 0 шт
18 - 23: 0%, 0 шт
23 - 30: 8%, 1 шт
30 - 40: 50%, 6 шт
40 - 50: 42%, 5 шт
50 - 60: 0%, 0 шт
60+: 0%, 0 шт

Сфера работы
IT сфера (разработчик, тестировщик, администратор и т.п.): 0%, 0 шт
Профессии связанные с числами (аналитик, бухгалтер, инженер и т.п.): 0%, 0 шт
Гуманитарий (общение с людьми, искусство, медицина и т.д.): 33%, 4 шт
Предприниматель, руководитель: 50%, 6 шт
Преподаватель, учитель: 17%, 2 шт
Школьник, студент: 0%, 0 шт
Пенсионер: 0%, 0 шт
Декрет: 0%, 0 шт

Доход
0 руб.: 0%, 0 шт
до 30 000 руб.: 0%, 0 шт
до 60 000 руб.: 0%, 0 шт
до 100 000 руб.: 25%, 3 шт
более 100 000 руб.: 75%, 9 шт

Обучение
Да, если я точно найду работу после обучения.: 0%, 0 шт
Да, если это поможет мне в реализации проекта.: 100%, 12 шт
Нет. Хочу получить только бесплатные материалы.: 0%, 0 шт

Время на обучение
до 5 часов в неделю: 0%, 0 шт
до 10 часов